# Project: Performance Evaluation of Bandit Algorithms

- In this project, you will implement several classical bandit algorithms, evluate their performance via numerical comparison and finally gain inspiring intuition.

## Part I: Classical Bandit Algorithms

We consider a time-slotted bandit system ($t=1,2,\ldots$) with three arms.
We denote the arm set as $\{1,2,3\}$.
Pulling each arm $j$ ($ j \in \{1,2,3\}$) will obtain a random reward $r_{j}$, which follows a Bernoulli distribution with mean $\theta_{j}$, *i.e.*, Bern($\theta_{j}$).
Specifically,

\begin{equation*}
	\begin{aligned}
		r_{j} = 
		\begin{cases}
			1, & w.p.\ \theta_{j}, \\
			0, & w.p.\ 1-\theta_{j},			
		\end{cases}
	\end{aligned}
\end{equation*}
where $\theta_{j}, j \in\{1,2,3\}$ are parameters within $(0,1)$.
  
Now we run this bandit system for $N$ ($N \gg 3$) time slots.
In each time slot $t$, we choose one and only one arm from these three arms, which we denote as $I(t) \in \{1,2,3\}$.
Then we pull the arm $I(t)$ and obtain a random reward $r_{I(t)}$.
Our objective is to find an optimal policy to choose an arm $I(t)$ in each time slot $t$ such that the expectation of the aggregated reward over $N$ time slots is maximized, *i.e.*,

\begin{equation*}
	\begin{aligned}
		\max_{I(t),t = 1,\dots,N} \ \  \mathbb{E}\left[\sum_{t=1}^{N} r_{I(t)} \right].
	\end{aligned}  	
\end{equation*}

If we know the values of $\theta_{j},j \in \{1,2,3\}$, this problem is trivial.
Since $r_{I(t)} \sim \text{Bern}(\theta_{I(t)})$,

\begin{equation*}
	\begin{aligned}
		\mathbb{E}\left[\sum_{t=1}^N r_{I(t)} \right] 
		= \sum_{t=1}^{N} \mathbb{E}[r_{I(t)}] 
		= \sum_{t=1}^N \theta_{I(t)}.
	\end{aligned} 	
\end{equation*}

Let $I(t) = I^{*} = \mathop{\arg \max}\limits_{ j \in \{1,2,3\}} \ \theta_j$ for $t=1,2,\ldots,N$, then 

\begin{equation*}
	\begin{aligned}
		\max_{I(t),t=1,\ldots,N} \ \  \mathbb{E}\left[\sum_{t=1}^N r_{I(t)} \right] = N \cdot \theta_{I^*}.
	\end{aligned} 	
\end{equation*}

However, in reality, we do not know the values of $\theta_{j},j \in \{1,2,3\}$.
We need to estimate the values $\theta_{j}, j \in \{1,2,3\}$ via empirical samples, and then make the decisions in each time slot. 
Next we introduce three classical bandit algorithms: $\epsilon$-greedy, UCB, and TS, respectively.

### $\epsilon$-greedy Algorithm ($0 \leq \epsilon \leq 1$)
<img src="figures/e-greedy.jpg" width="50%" align='left'>

### UCB (Upper Confidence Bound) Algorithm
<img src="figures/UCB.jpg" width="50%" align='left'>

### TS (Thompson Sampling) Algorithm
<img src="figures/TS.jpg" width="50%" align='left'>

### Problems

1. Now suppose we obtain the parameters of the Bernoulli distributions from an oracle, which are shown in the following table. Choose $N=5000$ and compute the theoretically maximized expectation of aggregate rewards over $N$ time slots. We call it the oracle value. Note that these parameters $\theta_{j}, j \in \{1,2,3\}$ and oracle values are unknown to all bandit algorithms.

| Arm $j$ | 1   | 2   | 3   |
|---------|-----|-----|-----|
| $\theta_j$ | 0.7 | 0.5 | 0.4 |

**Your anwser of problem 1 in Part I**

Since each arm's parameter is oracled.

So we just need to choose the arm with the largest parameter to have the maximum expectation of aggregate rewards over $N$ time slots.

Since $\theta_1 = 0.7, \theta_2 = 0.5, \theta_3 = 0.4$,

so $\theta_1 > \theta_2 > \theta_3$,

so we choose arm 1 everytime.

i.e. $$\forall t, I(t)=I^*=\argmax_{j\in\{1,2,3\}}\theta_j=1$$
$$\theta_{I(t)} = \theta_1 = 0.7$$

Also, since $r_{I(t)} \sim \text{Bern}(\theta_{I(t)})$.

So $E(r_{I(t)}) = \theta_{I(t)}$.

So the maximum expected value is 
$$\max_{I(t),t=1,2,\cdots,N}\ E\big[\sum_{t=1}^Nr_{I(t)}\big]$$
$$=\max_{I(t),t=1,2,\cdots,N}\ \sum_{t=1}^NE\big[r_{I(t)}\big]$$
$$=N \cdot \theta_{I^*} = 5000 \times 0.7 = 3500$$

So above all, with the given oracle parameters, the maximum expected value is 3500.

2. Implement aforemented three classical bandit algorithms with following settings: 
   
	- $N=5000$
	- $\epsilon$-greedy with $\epsilon \in \{0.1, 0.5, 0.9\}$.
	- UCB with $c \in \{1,5,10\}$.
	- TS with
    	- $\left\{(\alpha_1,\beta_1)=(1,1),(\alpha_2,\beta_2)=(1,1),(\alpha_3,\beta_3)=(1,1)\right\}$ 
    	- $\left\{(\alpha_1,\beta_1)=(601,401),(\alpha_2,\beta_2)=(401,601),(\alpha_3,\beta_3)=(2,3)\right\}$

**Your anwser of problem 2 in Part I**

In [48]:
import matplotlib.pyplot as plt
import numpy as np
import random, math, copy
### Import more packages if you need

import tqdm
import matplotlib.pyplot as plt

In [28]:
### Feel free to insert more blocks or helper functions if you need.

# since the arm's index are {1,2,3}
# so we need to add a 0 at index 0
# to make the index of arm's count and theta match the arm's index

theta_oracled = [0, 0.7, 0.5, 0.4] # the oracled theta of each arm

count = None
theta = None

def init_greedy():
    global count, theta
    count = [0, 0, 0, 0] # the initial count of each arm 
    theta = [0, 0, 0, 0] # the initial theta of each arm

def init_UCB():
    global count, theta
    count = [0, 1, 1, 1] # the initial count of each arm 
    theta = [0, 0, 0, 0] # the initial theta of each arm
    for t in range(1, 4):
        arm = t
        count[arm] = 1
        r_i = np.random.binomial(1, theta_oracled[arm]) # R_I(t) ~ Bern(theta_oracled[I(t)])
        theta[arm] = r_i

In [51]:
### Implementation of epsilon-Greedy:
### n is the number of time slots, epsilon is the parameter of the algorithm
### return the total reward
def greedy(n, epsilon):
    global count, theta
    init_greedy() # initialize the count and theta of each arm

    for t in range(1, n + 1): # the time slot
        prob = random.random() # return value is in [0, 1)
        arm = None # the arm to be chosen
        if prob < epsilon: # explore (with probability epsilon)
            arm = random.randint(1, 3) # randomly choose an arm from {1,2,3}
        else: # exploit (with probability 1 - epsilon)
            arm = np.argmax(theta) # choose the best arm
            if arm == 0: # if this happened, it means that all the theta are 0
                # so we can randomly choose an arm from {1,2,3}
                arm = random.randint(1, 3) # randomly choose an arm from {1,2,3}
        
        # print("time slot: ", t, " arm: ", arm)

        r_i = np.random.binomial(1, theta_oracled[arm]) # r_i ~ Bern(theta_oracled[arm])

        count[arm] += 1 # update the count of the chosen arm
        theta[arm] += 1 / count[arm] * (r_i - theta[arm]) # update the theta of the chosen arm
    
    reward = count[1] * theta[1] + count[2] * theta[2] + count[3] * theta[3] # the expectation of the reward
    return reward, regret # return the total reward and regret

In [65]:
### Implementation of UCB Algorithm:
### n is the number of time slots, c is the parameter of the algorithm
### return the total reward
def UCB(n, c):
    global count, theta
    init_UCB() # initialize the count and theta of each arm
    
    for t in range(4, n + 1):
        arm = np.argmax([theta[i] + c * math.sqrt(2 * math.log(t) / count[i]) for i in range(1, 4)]) + 1
        r_i = np.random.binomial(1, theta_oracled[arm]) # r_i ~ Bern(theta_oracled[arm])
        
        count[arm] += 1 # update the count of the chosen arm
        theta[arm] += 1 / count[arm] * (r_i - theta[arm]) # update the theta of the chosen arm
    
    reward = count[1] * theta[1] + count[2] * theta[2] + count[3] * theta[3] # the expectation of the reward
    return reward # return the total reward

In [78]:
### Implementation of TS Algorithm
### n is the number of time slots, a and b are the parameters of the algorithm
### return the total reward
def TS(n, a, b):
    reward = 0 # the expectation of the reward

    for t in range(1, n + 1):
        for i in range(1, 4):
            theta[i] = np.random.beta(a[i], b[i]) # theta[i] ~ Beta(a[i], b[i])
        arm = np.argmax(theta[1:4]) + 1 # choose the best arm
        r_i = np.random.binomial(1, theta_oracled[arm]) # r_i ~ Bern(theta_oracled[arm])

        a[arm] += r_i # update a[arm]
        b[arm] += 1 - r_i # update b[arm]

        reward += r_i # update the expectation of the reward
    return reward

3. Regard each of the above setting in problem 2 of Part I as an experiment (in total $8$ experiments).
Run each experiment $200$ independent trials (change the random seed).
Plot the final result (in terms of rewards and regrets) averaged over these $200$ trials.

**Your anwser of problem 3 in Part I**

In [81]:
### Your code for problem 1.3. Feel free to insert more blocks or helper functions if you need.

N = 5000
repeat_time = 200

# 1. the epsilon-greedy algoithm
"""
epsilon = [0.1, 0.5, 0.9]
average = [0, 0, 0]

for i in range(3):
    for _ in tqdm.tqdm(range(repeat_time)):
        average[i] += greedy(N, epsilon[i]) / repeat_time

print(average)
"""

# 2. the UCB algorithm
"""

c = [1, 5, 10]
average = [0, 0, 0]
for i in range(3):
    for _ in tqdm.tqdm(range(repeat_time)):
        average[i] += UCB(N, c[i]) / repeat_time

print(average)
"""

a = [[0 ,1, 1, 1], [0, 601, 401, 2]]
b = [[0, 1, 1, 1], [0, 401, 601, 3]]
average = [0, 0]
for i in range(2):
    for _ in tqdm.tqdm(range(repeat_time)):
        average[i] += TS(N, a[i], b[i]) / repeat_time

print(average)
print(average)
print(average)


# TODO
# print the reward and the regret!!!!!

100%|██████████| 200/200 [00:19<00:00, 10.29it/s]

[3501.879999999999, 3496.2000000000007]
[3501.879999999999, 3496.2000000000007]
[3501.879999999999, 3496.2000000000007]


4. Compute the gaps between the algorithm outputs (aggregated rewards over $N$ time slots) and the oracle value. Compare the numerical results of $\epsilon$-greedy, UCB, and TS.
   - Which one is the best?
   - Discuss the impacts of $\epsilon$, $c$, and $\alpha_{j}$, $\beta_{j}$, respectively. 

**Your anwser of problem 4 in Part I**

In [ ]:
### Your code for problem 1.4. Feel free to insert more blocks or helper functions if you need.

5. Give your understanding of the exploration-exploitation trade-off in bandit algorithms.

**Your anwser of problem 5 in Part I**



Actually, initially I made some mistakes on understanding of the bandit algorithm.

$\hat{\theta}_j$ is your evaluation of $\Theta(j)$. In Bandit model, the paremeter of mean reward $\theta$ is unknown, and it decides the reward you obtain. What you can know is your evaluation of $\theta$ , which is $\hat{\theta}$ and you decide your choice according your evaluation $\hat{\theta}$

6. We implicitly assume the reward distribution of these three arms are independent. How about the dependent case?
	Can you design an algorithm to exploit such information to obtain a better result?

**Your anwser of problem 6 in Part I**

In [ ]:
### Your code for problem 1.6. Feel free to insert more blocks or helper functions if you need.

## Part II: Bayesian Bandit Algorithms

There are two arms which may be pulled repeatedly in any order.
Each pull may result in either a success or a failure.
The sequence of successes and failures which results from pulling arm $i$ ($i \in \{1, 2\}$) forms a Bernoulli process with unknown success probability $\theta_{i}$.
A success at the $t^{th}$ pull yields a reward $\gamma^{t-1}$ ($0 < \gamma <1$), while an unsuccessful pull yields a zero reward.
At time zero, each $\theta_{i}$ has a Beta prior distribution with two parameters $\alpha_{i}, \beta_{i}$ and these distributions are independent for different arms.
These prior distributions are updated to posterior distributions as arms are pulled.
Since the class of Beta distributions is closed under Bernoulli sampling, posterior distributions are all Beta distributions.
How should the arm to pull next in each time slot be chosen to maximize the total expected reward from an infinite sequence of pulls?

1. 	One intuitive policy suggests that in each time slot we should pull the arm for which the current expected value of $\theta_{i}$ is the largest.
	This policy behaves very good in most cases.
	Please design simulations to check the behavior of this policy.

**Your anwser of problem 1 in Part II**

In [1]:
### Your code for problem 2.1. Feel free to insert more blocks or helper functions if you need.

2. However, such intuitive policy is unfortunately not optimal.
	Please provide an example to show why such policy is not optimal. 

**Your anwser of problem 2 in Part II**

TODO

Please provide an example to show why such policy is not optimal






3. For the expected total reward under an optimal policy, show that the following recurrence equation holds:

\begin{equation*}
		\begin{aligned}
			R_{1}(\alpha_{1},\beta_{1}) 
			= & \frac{\alpha_{1}}{\alpha_{1}+\beta_{1}} [1+\gamma R(\alpha_{1} + 1, \beta_{1}, \alpha_{2}, \beta_{2})] \\
				& + \frac{\beta_{1}}{\alpha_{1} + \beta_{1}} [\gamma R(\alpha_{1}, \beta_{1} + 1, \alpha_{2}, \beta_{2})]; \\
			R_{2}(\alpha_{2}, \beta_{2}) 
			= & \frac{\alpha_{2}}{\alpha_{2} + \beta_{2}} [1 + \gamma R(\alpha_{1}, \beta_{1}, \alpha_{2} + 1, \beta_{2})] \\
				& + \frac{\beta_{2}}{\alpha_{2} + \beta_{2}} [\gamma R(\alpha_{1}, \beta_{1}, \alpha_{2}, \beta_{2} + 1)]; \\
			R(\alpha_{1}, \beta_{1}, \alpha_{2}, \beta_{2}) 
			= & \max \left\{ R_{1}(\alpha_{1}, \beta_{1}), R_{2}(\alpha_{2}, \beta_{2}) \right\}.
		\end{aligned}  	
	\end{equation*}

**Your anwser of problem 3 in Part II**

TODO

prove the equation







4. For the above equations, how to solve it exactly or approximately? 

**Your anwser of problem 4 in Part II**

In [ ]:
### Your code for problem 2.4 if needed.

5. Find the optimal policy.

**Your anwser of problem 5 in Part II**

In [ ]:
### Your code for problem 2.5. Feel free to insert more blocks or helper functions if you need.